<a href="https://colab.research.google.com/github/FS75/ict3104-team01-2023/blob/T01-43-Select-5-trained-models-and-obtain-testing-results/project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 🕺🕺🕺 Install Environment

In [1]:
# !update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.8 2
# !update-alternatives --install /usr/local/bin/python3 python3 /usr/bin/python3.9 1
# !python --version
# !apt-get update
# !apt install software-properties-common
# !sudo dpkg --remove --force-remove-reinstreq python3-pip python3-setuptools python3-wheel
# !apt-get install python3-pip

print('Git clone project and install requirements...')
!rm -rf ict3104-team01-2023 && git clone https://github.com/FS75/ict3104-team01-2023.git
# %cd ict3104-team01-2023
# !git checkout T01-27-choose-a-dataset-subfolder
# %cd ..
!rm -rf mmpose && git clone https://github.com/open-mmlab/mmpose.git

Git clone project and install requirements...
Cloning into 'ict3104-team01-2023'...
remote: Enumerating objects: 1349, done.
remote: Counting objects: 100% (136/136), done.
remote: Compressing objects: 100% (129/129), done.
remote: Total 1349 (delta 19), reused 86 (delta 7), pack-reused 1213
Receiving objects: 100% (1349/1349), 3.40 GiB | 29.07 MiB/s, done.
Resolving deltas: 100% (149/149), done.
Updating files: 100% (1014/1014), done.
Cloning into 'mmpose'...
remote: Enumerating objects: 30276, done.
remote: Counting objects: 100% (1052/1052), done.
remote: Compressing objects: 100% (653/653), done.
remote: Total 30276 (delta 513), reused 803 (delta 390), pack-reused 29224
Receiving objects: 100% (30276/30276), 52.77 MiB | 23.88 MiB/s, done.
Resolving deltas: 100% (21323/21323), done.


In [ ]:
!cd /content/ict3104-team01-2023/FollowYourPoseTeam1
!export PYTHONPATH=/content/ict3104-team01-2023/FollowYourPoseTeam1:$PYTHONPATH
# %pip install -r requirements.txt
!python3.10 -m pip install -q -U --pre triton
!apt update
!python3.10 -m pip install -q diffusers==0.11.1 torch==1.13.1 transformers==4.26.0 bitsandbytes==0.35.4 \
imageio-ffmpeg xformers==0.0.16  --extra-index-url https://download.pytorch.org/whl/cu113


Hit:1 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
29 packages can be upgraded. Run 'apt list --upgradable' to see them.


In [ ]:
%cd /content/ict3104-team01-2023/FollowYourPoseTeam1
!python3.10 -m pip install -r requirements.txt

/content/ict3104-team01-2023/FollowYourPoseTeam1
  Using cached transformers-4.26.1-py3-none-any.whl (6.3 MB)
  Using cached triton-2.0.0-1-cp310-cp310-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (63.3 MB)
  Attempting uninstall: transformers
    Found existing installation: transformers 4.26.0
    Uninstalling transformers-4.26.0:
      Successfully uninstalled transformers-4.26.0
  Attempting uninstall: triton
    Found existing installation: triton 2.1.0
    Uninstalling triton-2.1.0:
      Successfully uninstalled triton-2.1.0


## 💃💃💃 Weight

In [ ]:
%mkdir checkpoints
!git lfs install
!git clone https://huggingface.co/YueMafighting/FollowYourPose_v1
%mv /content/ict3104-team01-2023/FollowYourPoseTeam1/FollowYourPose_v1/* /content/ict3104-team01-2023/FollowYourPoseTeam1/checkpoints/
%rm -rf FollowYourPose_v1

Updated git hooks.
Git LFS initialized.
Cloning into 'FollowYourPose_v1'...
remote: Enumerating objects: 42, done.
remote: Total 42 (delta 0), reused 0 (delta 0), pack-reused 42
Unpacking objects: 100% (42/42), 584.62 KiB | 5.85 MiB/s, done.
Filtering content: 100% (9/9), 9.75 GiB | 35.58 MiB/s, done.


## 🕺🕺🕺 Sample

Due to memory of GPU, we recommend set video_length=8 in ./config/pose_sample.yaml for running successfully.

Meanwhile, we should keep the skeleton frame length(./followyourpose/pipelines/pipeline_followyourpose.py:422 ) equal with video_length

In [ ]:
!pip install tensorrt
%cd /content/ict3104-team01-2023/FollowYourPoseTeam1
!pwd
!TORCH_DISTRIBUTED_DEBUG=DETAIL accelerate launch txt2video.py --config="configs/pose_sample.yaml"  --skeleton_path="./pose_example/vis_ikun_pose2.mov"

{'prediction_type'} was not found in config. Values will be initialized to default values.
{'prediction_type'} was not found in config. Values will be initialized to default values.
load from checkpoint ./checkpoints/followyourpose_checkpoint-1000
10/27/2023 11:39:57 - INFO - accelerate.accelerator - Loading states from ./checkpoints/followyourpose_checkpoint-1000
10/27/2023 11:40:17 - INFO - accelerate.checkpointing - All model weights loaded successfully
10/27/2023 11:40:17 - INFO - accelerate.checkpointing - All optimizer states loaded successfully
10/27/2023 11:40:17 - INFO - accelerate.checkpointing - All scheduler states loaded successfully
10/27/2023 11:40:17 - INFO - accelerate.checkpointing - Could not load random states
10/27/2023 11:40:17 - INFO - accelerate.accelerator - Loading in 0 custom states
100% 50/50 [04:06<00:00,  4.93s/it]
Video loaded successfully: /content/ict3104-team01-2023/FollowYourPoseTeam1/checkpoints/inference/sample-1000-33-2023-10-27 11:40:17.227680/Iro

## 💃💃💃 The videos are output into ./checkpoints/inference/sample-xxx/<prompt>.gif

##**Data Exploration Section** 🔍🔍🔍
####This section loads, lists and displays video data from the Charades project. Here you can view the video playback of your chosen file.
-------------------
  
####How to use:
1. Click "Run Cell", the play button on the left
2. Select a video from the dropdown menu.
3. Click "Play Video"

In [ ]:
# @title
import ipywidgets as widgets
from IPython.display import display
from IPython.display import HTML
from moviepy.editor import *
from IPython.display import clear_output

!ls
def play_video(change):
    clear_output(wait=True)  # clear the current cell output before displaying new video
    if file_selector.value:
        # Split the value into folder and filename
        folder, filename = file_selector.value.split(" - ", 1)
        video_path = os.path.join('/content/ict3104-team01-2023/data/videos', folder, filename)
        video_clip = VideoFileClip(video_path)

        # Resize the video to the desired dimensions
        video_resized = video_clip.resize(newsize=(512, 512))

        # Display the video
        display(file_selector)  # re-display the dropdown
        display(play_button)  # re-display the play button
        # Display the resized video
        display(video_resized.ipython_display())
    else:
        print("Please select a video")

# Base path
base_path = '/content/ict3104-team01-2023/data/videos'

# Subfolders list
subfolders = ['charades', 'youtube']

# Collect all .mp4 files from the subfolders with headers
files = []
for subfolder in subfolders:
    subfolder_path = os.path.join(base_path, subfolder)
    for f in os.listdir(subfolder_path):
        if os.path.isfile(os.path.join(subfolder_path, f)) and f.endswith('.mp4'):
            files.append(f"{subfolder} - {f}")

# Create a dropdown menu with the file names
file_selector = widgets.Dropdown(
    options=files,
    description='Select Video:',
    disabled=False,
)

# Create a button to play the video
play_button = widgets.Button(description="Play Video")

# Attach the play_video function to the button's on_click event
play_button.on_click(play_video)

# Display the dropdown menu and play button
display(file_selector)
display(play_button)

app_followyourpose.py  example_video.zip	    LICENSE	      requirements.txt
app.py		       followyourpose		    multi_train.sh    sample.sh
checkpoints	       gif_results		    pose_example      train_followyourpose.py
configs		       inference_followyourpose.py  quick_demo.ipynb  txt2video.py
example.py	       inference_mmpose.py	    README.md


Dropdown(description='Select Video:', options=('charades - 0CVZN.mp4', 'charades - 0CX32.mp4', 'charades - 0A8…

Button(description='Play Video', style=ButtonStyle())

## **Inference Section** 💡💡💡
####This section allows you to perform inference using a pretrained genAI model based on the Charades project.  
-------------------
  
####How to use (Please Add):
1.
2.
3.

### Customisation of configs (aka models)

In [2]:
pip install ruamel.yaml

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 113.7/113.7 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 526.7/526.7 kB 11.4 MB/s eta 0:00:00


In [13]:
import ipywidgets as widgets
from IPython.display import display, clear_output
import os
import shutil
from collections import OrderedDict
import ruamel.yaml

yaml = ruamel.yaml.YAML()
yaml.preserve_quotes = True
yaml.indent(mapping=2, sequence=4, offset=2)

config_template = 'ict3104-team01-2023/FollowYourPoseTeam1/configs/pose_sample_template.yaml'

# Define a custom layout for the input widget
input_layout = widgets.Layout(width='500px')

button_layout = widgets.Layout(width='250px')

# Create input widgets with the custom layout
file_name_input = widgets.Text(
    value='',
    placeholder='Config name here (no .yaml in string)',
    description='Config name:',
    disabled=False,
    layout=input_layout
)

video_length_input = widgets.IntText(
    value=8,
    description='Video Length:',
    disabled=False,
    layout=input_layout
)

width_input = widgets.IntText(
    value=512,
    description='Width:',
    disabled=False,
    layout=input_layout
)

height_input = widgets.IntText(
    value=512,
    description='Height:',
    disabled=False,
    layout=input_layout
)

# Container to hold dynamically created prompt input fields
prompt_inputs = [
    widgets.Text(
        value='',
        placeholder='Type your prompt here',
        description='Prompt 1:',
        disabled=False,
        layout=input_layout
    )
]

# Function to handle the click of "Add Prompt" button
def add_prompt(button):
    # Create a new prompt input field
    new_prompt = widgets.Text(
        value='',
        placeholder='Type your prompt here',
        description=f'Prompt {len(prompt_inputs) + 1}:',
        disabled=False,
        layout=input_layout
    )
    prompt_inputs.append(new_prompt)

    # Redraw all UI elements
    redraw_ui()

# Create an "Add prompt" button
add_prompt_button = widgets.Button(description="Add Prompt", layout=button_layout)
add_prompt_button.on_click(add_prompt)

def remove_latest_prompt(button):
    if len(prompt_inputs) > 1:  # To ensure there's always at least one prompt
        prompt_inputs.pop()
        redraw_ui()

remove_prompt_button = widgets.Button(description="Remove Latest Prompt", layout=button_layout)
remove_prompt_button.on_click(remove_latest_prompt)

def copy_file(button):
    # Check for config name and at least one valid prompt
    if not file_name_input.value or not any(prompt.value for prompt in prompt_inputs):
        clear_output(wait=True)
        redraw_ui()
        print("Please specify a config name and at least one prompt.")
        return

    # Redraw UI
    redraw_ui()

    # Check if the template file exists
    if os.path.exists(config_template):
        with open(config_template, 'r') as f:
            content = yaml.load(f)

        # Update the yaml content with input values
        content['validation_data']['video_length'] = video_length_input.value
        content['validation_data']['width'] = width_input.value
        content['validation_data']['height'] = height_input.value
        content['validation_data']['prompts'] = [prompt.value for prompt in prompt_inputs if prompt.value]

        # Get the user's input for the file name
        dest_file_name = file_name_input.value + ".yaml"

        # Define the destination path for the copied file
        dest_path = 'ict3104-team01-2023/data/pre_trained_models/' + dest_file_name

        # Write the updated content to the new file
        with open(dest_path, 'w') as f:
            yaml.dump(content, f)

        print(f"YAML file {dest_file_name} generated at: {dest_path}")
    else:
        print("Template file not found!")


copy_button = widgets.Button(description="Generate Config", layout=button_layout)
copy_button.on_click(copy_file)

def redraw_ui():
    clear_output(wait=True)
    display(file_name_input, video_length_input, width_input, height_input)
    for prompt in prompt_inputs:
        display(prompt)
    display(add_prompt_button, remove_prompt_button, copy_button)

# Display the initial UI components
redraw_ui()


Text(value='yes', description='Config name:', layout=Layout(width='500px'), placeholder='Config name here (no …

IntText(value=8, description='Video Length:', layout=Layout(width='500px'))

IntText(value=512, description='Width:', layout=Layout(width='500px'))

IntText(value=512, description='Height:', layout=Layout(width='500px'))

Text(value='asd', description='Prompt 1:', layout=Layout(width='500px'), placeholder='Type your prompt here')

Button(description='Add Prompt', layout=Layout(width='250px'), style=ButtonStyle())

Button(description='Remove Latest Prompt', layout=Layout(width='250px'), style=ButtonStyle())

Button(description='Generate Config', layout=Layout(width='250px'), style=ButtonStyle())

### Selecting a pre-trained model

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import os

# Get the current working directory
# current_working_directory = os.getcwd()

# Print the current working directory
# print(current_working_directory)

# Assuming your folder path is '/
folder_path = '/content/ict3104-team01-2023/FollowYourPoseTeam1/configs/'

# List all files in the folder
files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# Create a dropdown menu with the file names
file_selector = widgets.Dropdown(
    options=files,
    description='Select Model:',
    disabled=False,
)

# Display the dropdown menu
display(file_selector)

# Now, the selected file can be accessed using file_selector.value

Dropdown(description='Select Model:', options=('pose_sample_template.yaml', 'pose_train.yaml', 'pose_sample.ya…

### For Input

In [ ]:
#@markdown Select video After running the code
import os
import ipywidgets as widgets
from IPython.display import display, HTML

# Base path
base_path = '/content/ict3104-team01-2023/data/videos'

# Subfolders list
subfolders = ['charades', 'youtube', "Cook", "Drink", "Drink2", "Drink3", "Eat", "GetupSitdown"
, "GetupSitdown2", "Readbook", "Usecomputer", "Usephone", "Usetablet", "Usetablet2", "Walk", "WatchTV"]

mp4_files = []
for subfolder in subfolders:
    subfolder_path = os.path.join(base_path, subfolder)
    for f in os.listdir(subfolder_path):
        if os.path.isfile(os.path.join(subfolder_path, f)) and (f.endswith('.mp4') or f.endswith('.avi')) :
            mp4_files.append(f"{subfolder} - {f}")

dropdown = widgets.Dropdown(
    options=mp4_files,
    description='Select video:',
    disabled=False,
)
def update_link(change):
    selected_mp4_file  = dropdown.value
    link = f'<a href="{selected_mp4_file}" target="_blank">{selected_mp4_file}</a>'
    link_output.value = link

dropdown.observe(update_link, 'value')

# Display the dropdown widget
display(dropdown)

# Create an output widget to display the selected directory as a link
link_output = widgets.Output()
display(link_output)

#@markdown Enter the things to show in the video
TEXT_Prompts = "dsa" #@param {type:"string"}


Dropdown(description='Select video:', options=('charades - 0CVZN.mp4', 'charades - 0CX32.mp4', 'charades - 0A8…

Output()

##**Training Section** 🏋🏼‍♂️🏋🏼‍♂️🏋🏼‍♂️
####This section allows you to train a new genAI model based on the Charades project.
-------------------
  
####How to use:
1. Run the first two cells below, then the third cell
  1. Take note to restart the run time before running the third cell
  2. Click on runtime from the menubar options
  3. Click on restart runtime
  4. Wait for system to complete restart before running third cell
2. Select the video you wish to use to extract a skeleton from
3. Click Generate and wait, the output cell will inform you when it has finished generating



In [ ]:
!pip install openmim
!pip install tqdm
!mim install mmengine
!mim install "mmcv>=2.0.1"
!mim install "mmdet>=3.0.0"
!rm -rf mmpose && git clone https://github.com/open-mmlab/mmpose.git

Looking in links: https://download.openmmlab.com/mmcv/dist/cu117/torch1.13.0/index.html
Looking in links: https://download.openmmlab.com/mmcv/dist/cu117/torch1.13.0/index.html
Looking in links: https://download.openmmlab.com/mmcv/dist/cu117/torch1.13.0/index.html
Cloning into 'mmpose'...
remote: Enumerating objects: 30276, done.
remote: Counting objects: 100% (1052/1052), done.
remote: Compressing objects: 100% (642/642), done.
remote: Total 30276 (delta 516), reused 812 (delta 401), pack-reused 29224
Receiving objects: 100% (30276/30276), 53.58 MiB | 26.84 MiB/s, done.
Resolving deltas: 100% (21324/21324), done.


In [ ]:
import os
os.chdir('/content/mmpose')
!pip install -e .
import mmpose

Obtaining file:///content/mmpose
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/jin-s13/xtcocoapi to /tmp/pip-install-kqbbjr5u/xtcocotools_7694407f8a0f426cbd528c2cd5c57459
  Running command git clone --filter=blob:none --quiet https://github.com/jin-s13/xtcocoapi /tmp/pip-install-kqbbjr5u/xtcocotools_7694407f8a0f426cbd528c2cd5c57459
  Resolved https://github.com/jin-s13/xtcocoapi to commit d74033ff1635e9002133b2380862bc2b728584d2
  Preparing metadata (setup.py) ... done
  Attempting uninstall: mmpose
    Found existing installation: mmpose 1.2.0
    Uninstalling mmpose-1.2.0:
      Successfully uninstalled mmpose-1.2.0
  Running setup.py develop for mmpose


###Choose a video and click Generate to start training process/generate skeleton.

<b>Note: only .mp4 videos are supported.</b>
<b>To train multiple videos at once, click the checkbox, and choose the folder in which multiple videos reside.</b>

Blue cube means that the training is still in progress  
Green cube means that the training is completed  
The skeleton can be found in mmpose/vis_results

In [ ]:
# @title
# #restart runtime before running this code for import MMPose to work
import os
import ipywidgets as widgets
from IPython.display import display
from IPython.display import HTML
from IPython.display import clear_output
from moviepy.editor import *
from mmpose.apis import MMPoseInferencer
from tqdm.notebook import tqdm

# Base path
base_path = '/content/ict3104-team01-2023/data/videos'

# Subfolders list
subfolders = [folder for folder in os.listdir(base_path)]

# Collect all .mp4 files from the subfolders with headers
files = [],
for subfolder in subfolders:
    subfolder_path = os.path.join(base_path, subfolder)
    for f in os.listdir(subfolder_path):
        if os.path.isfile(os.path.join(subfolder_path, f)) and f.endswith('.mp4'):
            files.append(f"{subfolder} - {f}")

# Create a checkbox...
multiple_videos = widgets.Checkbox(
    value=True,
    description='Generate Multiple Videos?',
    disabled=False,
    indent=False
)

# Create a dropdown menu with the folder names
folder_selector = widgets.Dropdown(
    options=subfolders,
    description='Select Folder:',
    disabled=False,
)

# Create a dropdown menu with the file names
file_selector = widgets.Dropdown(
    options=files,
    description='Select Video:',
    disabled=False,
)

visuals_dir = '/content/mmpose/vis_results'
os.makedirs(visuals_dir, exist_ok=True)
inferencer = MMPoseInferencer('human')

# def generate_skeleton(change):
#   folder, filename = file_selector.value.split(" - ", 1)
#   img_path = os.path.join('/content/ict3104-team01-2023/data/videos', folder, filename)
#   result_generator = inferencer(img_path, black_background = True,vis_out_dir='/content/mmpose/vis_results')
#   clear_output(wait=True)
#   print(filename+" selected.")
#   # Initialize tqdm in manual mode
#   pbar = tqdm(total=None, desc="Training progress", unit="item", position=0, leave=True)
#   # Iterate over result_generator and store results
#   results = []
#   for result in result_generator:
#       results.append(result)
#       #Progress bar won't grow as we do not know the total count of iterations
#       pbar.update(1)
#   pbar.close()
#   #Redisplay UI
#   clear_output(wait=True)
#   print(filename+" Skeleton Generated.")
#   display(file_selector)
#   display(gen_button)

def generate(folder, filename):
    img_path = os.path.join('/content/ict3104-team01-2023/data/videos', folder, filename)
    result_generator = inferencer(img_path, black_background = True,vis_out_dir='/content/mmpose/vis_results')
    clear_output(wait=True)
    print(filename+" selected.")
    # Initialize tqdm in manual mode
    pbar = tqdm(total=None, desc="Training progress", unit="item", position=0, leave=True)
    # Iterate over result_generator and store results
    results = []
    for result in result_generator:
        results.append(result)
        #Progress bar won't grow as we do not know the total count of iterations
        pbar.update(1)
    pbar.close()


def generate_skeleton(change):
    clear_output(wait=True)
    if (multiple_videos.value):
        folder_path = os.path.join(base_path, folder_selector.value)
        for filename in os.listdir(folder_path):
              if filename.endswith('.mp4'):
                  folder = folder_selector.value
                  generate(folder, filename)
    else:
        folder, filename = file_selector.value.split(" - ", 1)
        generate(folder, filename)

    #Redisplay UI
    clear_output(wait=True)
    print(filename+" Skeleton Generated.")
    display(file_selector)
    display(gen_button)

# clear_output(wait=True)
# Create a button to play the video
gen_button = widgets.Button(description="Generate MMPose Skeleton")
# Attach the play_video function to the button's on_click event
gen_button.on_click(generate_skeleton)

# Display the dropdown menu and play button
display(multiple_videos)
display(folder_selector)
display(file_selector)
display(gen_button)

KeyError: ignored

##**Testing Section** 📝📝📝
####This section allows you to evaluate a trained model based on the Charades project.
-------------------
  
####How to use (Please Add):
1.
2.
3.

###Load a pretrained model

In [ ]:
import ipywidgets as widgets
from IPython.display import display
import os

# Get the current working directory
# current_working_directory = os.getcwd()

# Print the current working directory
# print(current_working_directory)

# Assuming your folder path is '/
folder_path = 'ict3104-team01-2023/FollowYourPoseTeam1/configs/'

# List all files in the folder
files = [f for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

# Create a dropdown menu with the file names
file_selector = widgets.Dropdown(
    options=files,
    description='Select Model:',
    disabled=False,
)

# Display the dropdown menu
display(file_selector)

# Now, the selected file can be accessed using file_selector.value

###Progress Bar of testing done (Whoever doing the actual testing need to add this progress bar, if unsure lemme know -chun)

In [ ]:
from tqdm.notebook import tqdm
# Initialize tqdm in manual mode, pbar will be your progress bar
pbar = tqdm(total=None, desc="Training progress", unit="item", position=0, leave=True)
# Iterate over result_generator and store results
results = []
for result in result_generator:
    results.append(result)
    #Progress bar will increment by 1 every loop, as long as number is incrementing it means that work is being done
    pbar.update(1)
pbar.close()
#Optional step below, you may want to clear the output after the work is done as the text will be left behind and
#the user has to scroll all the way down to see any additional messages u would like to add
clear_output(wait=True)